In [7]:
import sys
sys.path.append('../..')
from sqlalchemy import create_engine, MetaData, select, Table
import schedule
import pandas as pd
from datetime import datetime
import time
from config import POSTGRES_UTEA
import math

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

metadata = MetaData()
reporte_tbl = Table("reporte", metadata, autoload_with=ENGINE, schema="datos_iag")
msj_whatsapp_tbl = Table("msj_whatsapp", metadata, autoload_with=ENGINE, schema="notificaciones_wsp")

In [8]:
def crear_mensaje(datos):    
    try:
        with ENGINE.begin() as conn:
            conn.execute(msj_whatsapp_tbl.insert(), datos.to_dict(orient='records'))
        print("✅ Se ha actualizado ")
    except Exception as e:
        print("❌ Error al insertar en tabla MSJ WHASTAPP", e)
    #return df

In [9]:
df = pd.read_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\CONTACTOS_GRUPO.xlsx')

In [11]:
df = df[df['area']>0]
df

,COD_COS,NOMBRE,NUM,OBS,area
0,40,FERNANDO MENDIETA - ENCARGADO,77603832,NaN,93.186306
1,235,FREDI AMURRIO TAPIA,78560631,NaN,80.444534
2,398,RENNY VARGAS,77340813,NaN,77.146766
3,196,"JUAN, ENCARGADO",69111108,NaN,67.904659
4,196,WIDEN GUTIERREZ,69111101,NaN,67.904659
5,393,LEOPOLDO AYALA,67840458,NaN,63.931285
6,138,WILMER GUZMAN,78171001,NaN,62.386155
7,46,MAURICIO ROCA,78555148,NaN,57.047577
8,179,ROBERTO,72207658,OBS,56.507643
9,452,GERSON MOISES CORDOVA,78061560,NaN,38.944154


In [23]:
df['fecha_registro'] = datetime.now()
df['cod_canero'] = df['NOMBRE']
df['cod_canero'] = df['COD_COS']
df['nombre_canero'] = df['NOMBRE']
df['numero_cel'] = df['NUM']

df['mensaje'] = df.apply(lambda row: f"""👉 *Estimado productor, le escribimos desde UTEA por lo siguiente:*

Según el 📡 monitoreo satelital del avance de cosecha 2025 se observa que aún tiene *{row['area']:.2f} hectáreas de caña hoja* en campo pendientes para cosechar.

🌱 Le recomendamos priorizar la cosecha de caña hoja lo antes posible, ya que dejarla más tiempo en campo puede afectar el desarrollo del rebrote y ocasionar menor producción en la siguiente zafra.

🙏 Agradecemos la recepción de este mensaje y su compromiso con el buen manejo de su cultivo. 💪 Estamos seguros de que seguir esta recomendación traerá mejores resultados para su producción. 🚜""", axis=1)


df['enviado'] = False
df['fecha_enviado'] = None
df['numero_contac'] = '591' + df['NUM'].astype(str) + '@s.whatsapp.net'

In [24]:
df = df.drop(['COD_COS', 'NOMBRE', 'NUM', 'OBS'], axis=1)

In [25]:
df

,area,fecha_registro,cod_canero,nombre_canero,numero_cel,mensaje,enviado,fecha_enviado,numero_contac
0,93.186306,2025-08-30 12:38:36.021114,40,FERNANDO MENDIETA - ENCARGADO,77603832,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59177603832@s.whatsapp.net
1,80.444534,2025-08-30 12:38:36.021114,235,FREDI AMURRIO TAPIA,78560631,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59178560631@s.whatsapp.net
2,77.146766,2025-08-30 12:38:36.021114,398,RENNY VARGAS,77340813,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59177340813@s.whatsapp.net
3,67.904659,2025-08-30 12:38:36.021114,196,"JUAN, ENCARGADO",69111108,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59169111108@s.whatsapp.net
4,67.904659,2025-08-30 12:38:36.021114,196,WIDEN GUTIERREZ,69111101,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59169111101@s.whatsapp.net
5,63.931285,2025-08-30 12:38:36.021114,393,LEOPOLDO AYALA,67840458,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59167840458@s.whatsapp.net
6,62.386155,2025-08-30 12:38:36.021114,138,WILMER GUZMAN,78171001,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59178171001@s.whatsapp.net
7,57.047577,2025-08-30 12:38:36.021114,46,MAURICIO ROCA,78555148,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59178555148@s.whatsapp.net
8,56.507643,2025-08-30 12:38:36.021114,179,ROBERTO,72207658,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59172207658@s.whatsapp.net
9,38.944154,2025-08-30 12:38:36.021114,452,GERSON MOISES CORDOVA,78061560,"👉 *Estimado productor, le escribimos desde UTE...",False,None,59178061560@s.whatsapp.net


In [26]:
crear_mensaje(df)

✅ Se ha actualizado 


In [47]:
enviar_reporte_genencia()
while True:
    schedule.run_pending()
    time.sleep(1)

✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 
✅ Se ha actualizado 


KeyboardInterrupt: 